*This notebook is intellectual property of Auquan and is distributed under the [Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International Public License](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode). Any modification or distribution of this notebook without express permission of Auquan is prohibited and will result in legal prosecution.*

# Fitting a Distribution

Now let's get back to our problem at hand. We want to model stock prices. Our job would become really easy if we could successfully fit a probability distribution to the returns of a stock.

Let's give this a try. We will take the returns of AAPL and try to fit a normal distribution to them. 

In [ ]:
# Install yahoo finance to obtain historical market data
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

startDateStr = '2007-12-01'
endDateStr = '2017-12-01'
data = yf.download('AAPL', startDateStr, endDateStr)
prices = data.Close

AAPL's daily close prices are loaded into a DataFrame `prices`. 

**Ex1: Now first, calculate the daily returns : $price(t)/price(t-1) - 1$**

The `DataFrame.shift()` function will be useful here

In [ ]:
# Calculate the daily returns
returns = None
returns = prices/prices.shift(1) - 1

Notice that last entry in returns df is NaN obviously. You can either drop it using `.dropna()` function or remember exclude the last value in any calculation. We'll do this for you in the next line 

In [ ]:
returns = returns.dropna()

Now plot a histogram of returns to eyeball their distribution:

In [ ]:
if returns:
    plt.figure(figsize=(8,8))
    plt.hist(returns, density=True, bins=500)
    plt.show()
else:
    print('Calculate daily returns correctly')

What do you think? Do the returns look normally distributed to you?

We will proceed from here assuming that the returns are normally distributed so that we can go through the steps of fitting a distribution. 
Next we calculate the sample mean and standard deviation of the series and see how a theoretical normal curve fits against the actual values. 

**Ex2: Calculate sample mean and sample standard deviation of returns. Pandas comes with built in functions, you can just use `.mean()` and `.std()` to calculate this**

In [ ]:
# Take the sample mean and standard deviation of the returns
sample_mean = None
sample_std_dev = None
print("Mean: ", sample_mean, " S Dev: ", sample_std_dev)

Now we plot an actual normal distribution with the same mean and standard deviation as that of AAPL returns. We overlay this on histogram of AAPL returns

In [ ]:
x = np.linspace(-(sample_mean + 4 * sample_std_dev), (sample_mean + 4 * sample_std_dev), len(returns))
sample_distribution = ((1/(sample_std_dev * 2 * np.pi)) * 
                       np.exp(-(x - sample_mean)*(x - sample_mean) / (2 * sample_std_dev * sample_std_dev)))
plt.hist(returns.values, bins = 500, density=True)
plt.plot(x, sample_distribution)
plt.xlabel('Value')
plt.ylabel('Occurrences');
plt.show()

Our theoretical curve for the returns has a substantially lower peak than the actual values. This makes sense because the returns are not actually normally distributed. 


### Why do we care about distributions?

Think back to previous exercise. We used a probabilistic framework to decide wether to trade or wait, the ideal price to trade etc. To be able to do this, we need to know the probabilities of stock moves. Now we obviously can't know the exact probabilites, be can make some estimates if you know the underlying distribution. 

A major reason why it is so difficult to model prices and returns is due to the underlying probability distributions. A lot of theories and frameworks in finance require that data be somehow related to the normal distribution. This is a major reason for why the normal distribution seems to be so prevalent. However, it is exceedingly difficult to find real-world data that fits nicely into the assumptions of normality. When actually implementing a strategy, you should not assume that data follows a distribution that it demonstrably does not unless there is a very good reason for it.

#### Normality Test

Now can we actually check whether the returns actually exhibit properties of a normal distribution analytically, without eyeballing? Ofcourse we can. There's a variety of statistical tests for this purpose. Here we will use the [Jarque-Bera test](http://www.statisticshowto.com/jarque-bera-test/), which indicates non-normality if the p-value is below a cutoff.

In [ ]:
import scipy.stats

#Set a cutoff
cutoff = 0.01

# Get the p-value of the normality test
k2, p_value = scipy.stats.mstats.normaltest(returns[:-1].values)
print("The JB test p-value is: ", p_value)
print("We reject the hypothesis that the data are normally distributed ", p_value < cutoff)
print("The skewness of the returns is: ", scipy.stats.skew(returns[:-1].values))
print("The kurtosis of the returns is: ", scipy.stats.kurtosis(returns[:-1].values))

The low p-value of the test leads us to *reject* the null hypothesis that the returns are normally distributed. This is due to the kurtosis of the normal distribution. The kurtosis of the normal distribution is $3$. A higher kurtosis leads to a higher peak. 

What was the kurtosis of your returns? (should be around 7) We'll talk about skew and kurtosis in the next post.


### Skew and Kurtosis for normal distribution:
1. **Skew**: It is a measure of the symmetry around the mean. A perfectly symmetrical data set will have a skewness of 0.   The normal distribution has a skewness of 0. Higher it is, more asymmetrical is the distribution around the mean.

2. **Kurtosis**: It is a measure of a peakedness around the mean compared to that around tails. The kurtosis decreases as the tails become lighter. Returns generally have heavy tails and hence high value of kurtosis.

Find more information [here](https://www.spcforexcel.com/knowledge/basic-statistics/are-skewness-and-kurtosis-useful-statistics)

Generally, when trying to fit a probability distribution to real-world values, we should have a particular distribution (or distributions) in mind. There are a variety of tests for different distributions that we can apply to see what might be the best fit. In additon, as more information becomes available, it will become necessary to update the sample mean and standard deviation or maybe even to find a different distribution to more accurately reflect the new information. The shape of the distribution will change accordingly.